In [73]:
import pandas as pd
from pathlib import Path
path = 'data/raw/natural-disasters/emdat_public_2022_12_22_full.xlsx'

In [74]:
disasters_all = pd.read_excel(path, skiprows = 6)

/Users/paulschmitt/PycharmProjects/tuw-python-2022WS/venv/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [75]:
disasters_all.head()

,Dis No,Year,Seq,Glide,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,...,"Reconstruction Costs, Adjusted ('000 US$)",Insured Damages ('000 US$),"Insured Damages, Adjusted ('000 US$)",Total Damages ('000 US$),"Total Damages, Adjusted ('000 US$)",CPI,Adm Level,Admin1 Code,Admin2 Code,Geo Locations
0,1900-9002-CPV,1900,9002,NaN,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.077091,NaN,NaN,NaN,NaN
1,1900-9001-IND,1900,9001,NaN,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.077091,NaN,NaN,NaN,NaN
2,1901-0003-BEL,1901,3,NaN,Technological,Technological,Industrial accident,Explosion,NaN,Coal mine,...,NaN,NaN,NaN,NaN,NaN,3.077091,NaN,NaN,NaN,NaN
3,1902-0012-GTM,1902,12,NaN,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,...,NaN,NaN,NaN,25000.0,781207.0,3.200175,NaN,NaN,NaN,NaN
4,1902-0003-GTM,1902,3,NaN,Natural,Geophysical,Volcanic activity,Ash fall,NaN,Santa Maria,...,NaN,NaN,NaN,NaN,NaN,3.200175,NaN,NaN,NaN,NaN


In [76]:
# Remove whitespaces from all col-names and convert them to lower-case
disasters_all.columns = [c.replace(' ', '_').lower() for c in disasters_all.columns]
disasters_all.columns

Index(['dis_no', 'year', 'seq', 'glide', 'disaster_group', 'disaster_subgroup',
       'disaster_type', 'disaster_subtype', 'disaster_subsubtype',
       'event_name', 'country', 'iso', 'region', 'continent', 'location',
       'origin', 'associated_dis', 'associated_dis2', 'ofda_response',
       'appeal', 'declaration', 'aid_contribution_('000_us$)', 'dis_mag_value',
       'dis_mag_scale', 'latitude', 'longitude', 'local_time', 'river_basin',
       'start_year', 'start_month', 'start_day', 'end_year', 'end_month',
       'end_day', 'total_deaths', 'no_injured', 'no_affected', 'no_homeless',
       'total_affected', 'reconstruction_costs_('000_us$)',
       'reconstruction_costs,_adjusted_('000_us$)',
       'insured_damages_('000_us$)', 'insured_damages,_adjusted_('000_us$)',
       'total_damages_('000_us$)', 'total_damages,_adjusted_('000_us$)', 'cpi',
       'adm_level', 'admin1_code', 'admin2_code', 'geo_locations'],
      dtype='object')

In [77]:
disasters_all = disasters_all[disasters_all.disaster_group == "Natural"]

In [78]:
relevant_col_names = ["year", "country", "iso", "disaster_type", "disaster_subtype", "total_deaths"]
disasters_all = disasters_all.filter(items=relevant_col_names)
disasters_all.rename(columns={'country':'country_name', 'iso':'country_code', 'disaster_subtype':'subtype', 'disaster_type':'type', 'total_deaths':'deaths'}, inplace=True)

In [79]:
disasters_global_attributes = ["year", "type", "subtype", "deaths"]
disasters_global = disasters_all[disasters_global_attributes]
disasters_global_filepath = Path('data/disaster-global.csv')
disasters_global.to_csv(filepath) 

NameError: name 'filepath' is not defined

In [ ]:
df.head(20)

In [ ]:
for col in df:
    print(col + ": " + str(df.loc[:, col].isnull().sum()))
print("Total: " + str(len(df)))

In [ ]:
un_country_codes = pd.read_csv("data/raw/country-codes/un-country-codes.csv", sep=";")
un_country_codes.columns = [c.replace(' ', '_').replace('-','_') for c in un_country_codes.columns]

In [ ]:
countries_with_iso = df.merge(un_country_codes, how="left", left_on='country_name', right_on='Country_or_Area')[["country_name", "country_code", "ISO_alpha3_Code"]]

In [ ]:
countries_with_iso.head()

In [ ]:
mismatches = countries_with_iso[countries_with_iso.ISO_alpha3_Code.isnull()]

In [ ]:
mismatches.country_name.unique()

In [ ]:
# Replace suffix
df['country_name'] = df['country_name'].apply(lambda x: x.replace(' (the)', ''))
# Reorder compley country-names
df['country_name'] = df['country_name'].apply(lambda x: x.split(',')[1] + " " + x.split(',')[0] if ',' in x else x)
# Remove random white-spaces at the start of a name
df['country_name'] = df['country_name'].apply(lambda x: x[1:] if x.startswith(' ') else x)


In [ ]:
new_country_names = {
    "Germany Fed Rep": "Germany",
    "Germany Dem Rep": "Germany",
    "Hong Kong": "China",
    "Macao": "China",
    "Åland": "Åland Islands",
    "Congo (the Democratic Republic of the)": "Congo",
    "Turkey": "Türkiye",
    "Korea (the Republic of)": "Republic of Korea",
    "Macedonia (the former Yugoslav Republic of)": "North Macedonia",
    "Congo (Democratic Republic of the)": "Democratic Republic of the Congo",
    "Yemen P Dem Rep": "Yemen",
    "Yemen Arab Rep": "Yemen",
    "Korea (the Democratic People's Republic of)": "Democratic People's Republic of Korea",
    "Korea (the Republic of)": "Republic of Korea",
    "Serbia Montenegro" : "Serbia",
    "Moldova (the Republic of)" : "Republic of Moldova",
    "Czech Republic" : "Czechia",
    "Taiwan (Province of China)" : "Taiwan"
    
}

In [ ]:
df = df.replace({"country_name": new_country_names}, inplace=False)

In [ ]:
countries_with_iso = df.merge(un_country_codes, how="left", left_on='country_code', right_on='ISO_alpha3_Code')[["country_name", "country_code", "ISO_alpha3_Code"]]
mismatches = countries_with_iso[countries_with_iso.ISO_alpha3_Code.isnull()]
mismatches.country_name.unique()

In [ ]:
countries_with_iso.head(10)
mask = countries_with_iso.notnull().all(axis=1)
countries_with_iso[~mask].country_name.unique()

In [ ]:
df.loc[df.country_name == "Germany", "country_code"] = "DEU"
df.loc[df.country_name == "Serbia", "country_code"] = "SRB"
df.loc[df.country_name == "Yemen", "country_code"] = "YEM"
df.loc[df.country_name == "Taiwan", "country_code"] = "TWM"
df.loc[df.country_name == "Canary Is", "country_code"] = "SPI"
df.loc[df.country_name == "Azores Islands", "country_code"] = "AZO"

In [ ]:
countries_with_iso = df.merge(un_country_codes, how="left", left_on='country_code', right_on='ISO_alpha3_Code')[["country_name", "country_code", "ISO_alpha3_Code"]]
mismatches = countries_with_iso[countries_with_iso.ISO_alpha3_Code.isnull()]
mismatches.country_name.unique()

In [ ]:
df[df.subtype.isnull()].head(20)

In [ ]:
filepath_country = Path('data/disaster-country.csv')
df.to_csv(filepath_country) 